In [1]:
import numpy as np
import pandas as pd
import constants
import functions
from functions import *
from constants import *
from importlib import reload

In [2]:
consants = reload(constants)
functions = reload(functions)

In [3]:
Po = np.zeros([ndt+1, ndx, ndy])
Pw = np.zeros([ndt+1, ndx, ndy])
Po[0] = Poi

In [4]:
Qo = np.zeros([ndt+1, ndx, ndy])
Qw = np.zeros([ndt+1, ndx, ndy])
Qo[0] = qo
Qw[0] = qw

In [5]:
S_wr = 0.2
S_or = 0.1
S_wi = 0.2

In [6]:
S_w = np.zeros([ndt+1, ndx, ndy])
S_o = np.zeros([ndt+1, ndx, ndy])
S_w[0] = S_wi
S_o[0] = 1 - S_wi

In [7]:
PHI = np.zeros([ndt, ndx, ndy])
Pc = np.zeros([ndt, ndx, ndy])
dPc = np.zeros([ndt, ndx, ndy])
dBo = np.zeros([ndt, ndx, ndy])
dBw = np.zeros([ndt, ndx, ndy])
ToW = np.zeros([ndt, ndx, ndy])
ToE = np.zeros([ndt, ndx, ndy])
ToN = np.zeros([ndt, ndx, ndy])
ToS = np.zeros([ndt, ndx, ndy])
TwW = np.zeros([ndt, ndx, ndy])
TwE = np.zeros([ndt, ndx, ndy])
TwN = np.zeros([ndt, ndx, ndy])
TwS = np.zeros([ndt, ndx, ndy])
Cpoo = np.zeros([ndt, ndx, ndy])
Cpow = np.zeros([ndt, ndx, ndy])
Cswo = np.zeros([ndt, ndx, ndy])
Csww = np.zeros([ndt, ndx, ndy])
Alpha = np.zeros([ndt, ndx, ndy])
Viso = np.zeros([ndt, ndx, ndy])
Visw = np.zeros([ndt, ndx, ndy])
Bo = np.zeros([ndt, ndx, ndy])
Bw = np.zeros([ndt, ndx, ndy])
t = 0

In [8]:
Pc[0], _ = capillary_pressure(S_wr, S_or, S_w[0], ndx, ndy)

Pw[0] = Po[0] - Pc[0]

In [9]:
Vis_o, Vis_w = viscosity(Po[0], Pw[0], ndx, ndy)

In [11]:
Bo, Bw, dBo, dBw = FVF(Po, Pw, Bo_0, Bw_0, Co, Cw, P_0, ndx, ndy, t)
Kr_o, Kr_w = relative_permeability(S_wr, S_or, S_w[0], ndx, ndy)
lambda_o, lambda_w = mobility(BCW, BCE, BCN, BCS, Kr_o, Kr_w, Vis_o, Vis_w, Bo, Bw, Po[0], Pw[0], ndx, ndy)

In [18]:
Jo = 2 * 3.14 * 1.127 * np.multiply(Kx, dz, lambda_o[0]) / (np.log(r_eq/rw_o) + S_o[0])
Jw = 2 * 3.14 * 1.127 * Kx * dz * (Bo / Bw * lambda_o[0] + lambda_w[0]) / (np.log(r_eq/rw_w) + S_w[0])
Jow = 2 * 3.14 * 1.127 * np.multiply(Kx, dz, lambda_w[0]) / (np.log(r_eq/rw_o) + S_o[0])

/tmp/ipykernel_2454/2077796487.py:1: RuntimeWarning: divide by zero encountered in true_divide
  Jo = 2 * 3.14 * 1.127 * np.multiply(Kx, dz, lambda_o[0]) / (np.log(r_eq/rw_o) + S_o[0])
/tmp/ipykernel_2454/2077796487.py:2: RuntimeWarning: divide by zero encountered in true_divide
  Jw = 2 * 3.14 * 1.127 * Kx * dz * (Bo / Bw * lambda_o[0] + lambda_w[0]) / (np.log(r_eq/rw_w) + S_w[0])
/tmp/ipykernel_2454/2077796487.py:3: RuntimeWarning: divide by zero encountered in true_divide
  Jow = 2 * 3.14 * 1.127 * np.multiply(Kx, dz, lambda_w[0]) / (np.log(r_eq/rw_o) + S_o[0])


In [42]:
BHPo

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0, 5500]])

In [41]:
if IBo == 1 and IBw == 1:
    BHPo = Po[0] + Qo[0] / Jo
    BHPw = Pw[0] + Qw[0] / Jw
    Qwo = Qo[0] * lambda_w[0] / lambda_o[0]
    for i in range(ndx):
        for j in range(ndy):
            if Qo[0][i][j] == 0:
                BHPo[0][i][j] = 0
            if Qw[0][i][j] == 0:
                BHPw[0][i][j] = 0
if IBo == 1 and IBw == 2:
    BHPo = Po[0] + Qo[0] / Jo
    Qw[0] = Jw * (-Pw[0] + BHPw)
    Qwo = Qo[0] * lambda_w[0] / lambda_o[0]
    for i in range(ndx):
        for j in range(ndy):
            if Qo[0][i][j] == 0:
                BHPo[0][i][j] = 0
            if BHPw[0][i][j] == 0:
                Qw[0][i][j] = 0
if IBo == 2 and IBw == 1:
    BHPw = Pw[0] + Qw[0] / Jw
    Qo[0] = (-Jo) * (Po[0] + BHPo)
    Qwo = (-Jow) * (Pw[0] - BHPo)
    for i in range(ndx):
        for j in range(ndy):
            if BHPo[0][i][j] == 0:
                Qo[0][i][j] = 0
            if Qw[0][i][j] == 0:
                BHPw[0][i][j] = 0
if IBo == 2 and IBw == 2:
    Qo[0] = (-Jo) * (Po[0] - BHPo)
    Qw[0] = (Jw) * (-Pw[0] + BHPw)
    Qwo = (-Jow) * (Pw[0] - BHPo)
    for i in range(ndx):
        for j in range(ndy):
            if BHPo[0][i][j] == 0:
                Qo[0][i][j] = 0
            if BHPw[0][i][j] == 0:
                Qw[0][i][j] = 0


/tmp/ipykernel_2454/3120544534.py:22: RuntimeWarning: invalid value encountered in true_divide
  BHPw = Pw[0] + Qw[0] / Jw


IndexError: invalid index to scalar variable.

In [37]:
Qo[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])